In [1]:
import tensorflow as tf
import numpy as np
import pickle

In [2]:
SIZE = 32
CHANNELS = 3
CLASSES = 10

L2_MULT = 0.008

In [3]:
data_path = "data/cifar"

def read_data(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding="bytes")
    return dict
def load_data():
    train_data_path = data_path + "/data_batch_"
    test_data_path = data_path + "/test_batch"
    
    train_data = read_data(train_data_path + "1")
    
    train_images = train_data[b'data']
    train_labels = train_data[b'labels']
    for i in range(2, 6):
        batch_data = read_data(train_data_path + str(i))
        
        train_images = np.concatenate((train_images, batch_data[b'data']))
        train_labels = np.concatenate((train_labels, batch_data[b'labels']))
        
    test_data = read_data(test_data_path)
    
    #train_images = [tf.image.per_image_standardization(np.reshape(img, [SIZE, SIZE, CHANNELS])) for img in train_images]
    #train_images = [tf.reshape(img, [SIZE, SIZE, CHANNELS]) for img in train_images]
    
    return train_images, train_labels, test_data

In [4]:
train_images, train_labels, test_data = load_data()

test_images = test_data[b'data']
test_labels = test_data[b'labels']

In [5]:
sess = tf.InteractiveSession()

In [6]:
flat_images = tf.placeholder(tf.uint8, [None, SIZE**2 * CHANNELS])
images = tf.cast(tf.reshape(flat_images, [-1, SIZE, SIZE, CHANNELS]), tf.float32)

labels = tf.placeholder(tf.int64, [None])

In [7]:
conv_summaries = []

In [8]:
conv1 = tf.layers.conv2d(images, 128, 5, 2, padding="VALID")
norm1 = tf.nn.lrn(conv1, 4, bias=1.0, alpha = 0.001 / 9.0, beta=0.75)

conv2 = tf.layers.conv2d(norm1, 128, 4, 1, padding="VALID")
norm2 = tf.nn.lrn(conv2, 4, bias=1.0, alpha = 0.001 / 9.0, beta=0.75)

conv3 = tf.layers.conv2d(norm2, 32, 4, 1, padding="SAME")
norm3 = tf.nn.lrn(conv3, 4, bias=1.0, alpha = 0.001 / 9.0, beta=0.75)

print(norm3.shape)

(?, 11, 11, 32)


In [9]:
flat_size = (11)**2 * 32

fc1 = tf.layers.dense(tf.reshape(norm3, [-1, flat_size]), 512)
output = tf.layers.dense(fc1, CLASSES)

In [10]:
cross_entro = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=output)
loss = tf.reduce_mean(cross_entro) + L2_MULT * tf.add_n([tf.nn.l2_loss(var) for var in tf.trainable_variables()])

train = tf.train.AdamOptimizer(1e-5, beta1=0.5).minimize(loss)

In [11]:
choices = tf.argmax(output, 1)
correct_choices = tf.equal(labels, choices)
accur = tf.reduce_mean(tf.cast(correct_choices, tf.float32))

In [12]:
tf.global_variables_initializer().run()

summary_writer = tf.summary.FileWriter("tmp/logs", sess.graph)

In [13]:
def generate_data(batch_size):
    indices = np.random.randint(0, len(train_images), [batch_size])
    return [train_images[i] for i in indices], [train_labels[i] for i in indices]

In [14]:
def calc_accur(data_images, data_labels):
    accur_list = []
    for i in range(10):#len(data_images) // TEST_BATCH_SIZE):
        batch_start = TEST_BATCH_SIZE * i
        batch_end = TEST_BATCH_SIZE * (i + 1)
        
        batch_images = data_images[batch_start:batch_end]
        batch_labels = data_labels[batch_start:batch_end]
        
        accur_list.append(accur.eval(feed_dict={flat_images: batch_images, labels: batch_labels}))
    return np.average(accur_list)

In [ ]:
BATCH_SIZE = 500
TEST_BATCH_SIZE = 500

for ITER in range(5000000):
    batch = generate_data(BATCH_SIZE)
    if ITER % 500 == 0:
        train_accur = 100 * calc_accur(train_images, train_labels)
        test_accur = 100 * calc_accur(test_images, test_labels)
        
        train_loss = loss.eval(feed_dict={flat_images: batch[0], labels: batch[1]})
        print("%05d Loss: %.2f | Training Accur: %.2f%% | Test Accur: %.2f%% " %(ITER, train_loss, train_accur, test_accur))
        
        for summary in conv_summaries:
            summary_writer.add_summary(summary.eval(feed_dict={flat_images:batch[0], labels: batch[1]}), ITER)
        summary_writer.flush()
        
    train.run(feed_dict={flat_images:batch[0], labels: batch[1]})


00000 Loss: 32.56 | Training Accur: 10.98% | Test Accur: 10.82% 
00500 Loss: 8.07 | Training Accur: 26.62% | Test Accur: 24.66% 
01000 Loss: 7.82 | Training Accur: 24.92% | Test Accur: 23.34% 
01500 Loss: 7.38 | Training Accur: 31.84% | Test Accur: 31.16% 
02000 Loss: 6.28 | Training Accur: 36.98% | Test Accur: 34.34% 
02500 Loss: 6.78 | Training Accur: 32.42% | Test Accur: 31.04% 
03000 Loss: 6.35 | Training Accur: 35.28% | Test Accur: 32.94% 
03500 Loss: 6.63 | Training Accur: 33.44% | Test Accur: 32.18% 
04000 Loss: 7.78 | Training Accur: 29.48% | Test Accur: 28.06% 
04500 Loss: 6.56 | Training Accur: 38.50% | Test Accur: 36.10% 
05000 Loss: 6.35 | Training Accur: 37.66% | Test Accur: 36.66% 
05500 Loss: 6.83 | Training Accur: 34.08% | Test Accur: 33.04% 
06000 Loss: 6.74 | Training Accur: 39.68% | Test Accur: 38.02% 
06500 Loss: 6.23 | Training Accur: 40.94% | Test Accur: 37.40% 
07000 Loss: 6.24 | Training Accur: 43.44% | Test Accur: 41.56% 
07500 Loss: 6.20 | Training Accur: 39.3

64000 Loss: 2.63 | Training Accur: 75.82% | Test Accur: 51.28% 
64500 Loss: 2.71 | Training Accur: 75.16% | Test Accur: 51.40% 
65000 Loss: 2.65 | Training Accur: 74.90% | Test Accur: 51.10% 
65500 Loss: 2.64 | Training Accur: 75.36% | Test Accur: 50.66% 
66000 Loss: 2.60 | Training Accur: 76.30% | Test Accur: 51.40% 
66500 Loss: 2.58 | Training Accur: 76.74% | Test Accur: 51.66% 
67000 Loss: 2.57 | Training Accur: 76.84% | Test Accur: 51.38% 
67500 Loss: 2.58 | Training Accur: 76.70% | Test Accur: 50.50% 
68000 Loss: 2.52 | Training Accur: 76.10% | Test Accur: 50.86% 
68500 Loss: 2.52 | Training Accur: 76.44% | Test Accur: 51.40% 
69000 Loss: 2.54 | Training Accur: 76.68% | Test Accur: 51.22% 
69500 Loss: 2.43 | Training Accur: 76.72% | Test Accur: 51.00% 
70000 Loss: 2.46 | Training Accur: 77.18% | Test Accur: 51.58% 
70500 Loss: 2.47 | Training Accur: 77.26% | Test Accur: 51.30% 
71000 Loss: 2.48 | Training Accur: 78.14% | Test Accur: 51.36% 
71500 Loss: 2.44 | Training Accur: 76.84

128000 Loss: 1.23 | Training Accur: 91.62% | Test Accur: 51.26% 
128500 Loss: 1.19 | Training Accur: 91.62% | Test Accur: 51.02% 
129000 Loss: 1.25 | Training Accur: 91.92% | Test Accur: 51.24% 
129500 Loss: 1.17 | Training Accur: 92.36% | Test Accur: 50.86% 
130000 Loss: 1.22 | Training Accur: 91.80% | Test Accur: 50.78% 
130500 Loss: 1.17 | Training Accur: 91.84% | Test Accur: 51.16% 
131000 Loss: 1.21 | Training Accur: 92.12% | Test Accur: 51.00% 
131500 Loss: 1.19 | Training Accur: 91.96% | Test Accur: 51.60% 
132000 Loss: 1.17 | Training Accur: 91.90% | Test Accur: 50.88% 
132500 Loss: 1.13 | Training Accur: 92.76% | Test Accur: 51.14% 
133000 Loss: 1.17 | Training Accur: 92.78% | Test Accur: 51.50% 
133500 Loss: 1.17 | Training Accur: 93.06% | Test Accur: 51.70% 
134000 Loss: 1.14 | Training Accur: 92.56% | Test Accur: 50.70% 
134500 Loss: 1.15 | Training Accur: 92.28% | Test Accur: 51.28% 
135000 Loss: 1.12 | Training Accur: 93.04% | Test Accur: 50.86% 
135500 Loss: 1.12 | Train

191500 Loss: 0.77 | Training Accur: 98.86% | Test Accur: 51.78% 
192000 Loss: 0.78 | Training Accur: 98.14% | Test Accur: 51.42% 
192500 Loss: 0.79 | Training Accur: 97.54% | Test Accur: 51.46% 
193000 Loss: 0.76 | Training Accur: 98.72% | Test Accur: 51.26% 
193500 Loss: 0.76 | Training Accur: 98.82% | Test Accur: 51.72% 
194000 Loss: 0.77 | Training Accur: 98.80% | Test Accur: 51.86% 
194500 Loss: 0.75 | Training Accur: 98.58% | Test Accur: 51.30% 
195000 Loss: 0.79 | Training Accur: 98.72% | Test Accur: 52.30% 
195500 Loss: 0.75 | Training Accur: 98.40% | Test Accur: 51.86% 
196000 Loss: 0.77 | Training Accur: 98.58% | Test Accur: 51.76% 
196500 Loss: 0.76 | Training Accur: 98.82% | Test Accur: 51.62% 
197000 Loss: 0.76 | Training Accur: 98.46% | Test Accur: 51.54% 
197500 Loss: 0.77 | Training Accur: 98.88% | Test Accur: 51.68% 
198000 Loss: 0.76 | Training Accur: 98.88% | Test Accur: 51.96% 
198500 Loss: 0.75 | Training Accur: 98.78% | Test Accur: 51.72% 
199000 Loss: 0.75 | Train

255000 Loss: 0.61 | Training Accur: 99.58% | Test Accur: 52.42% 
255500 Loss: 0.61 | Training Accur: 99.74% | Test Accur: 52.66% 
256000 Loss: 0.61 | Training Accur: 99.50% | Test Accur: 52.60% 
256500 Loss: 0.60 | Training Accur: 99.52% | Test Accur: 52.72% 
257000 Loss: 0.60 | Training Accur: 99.70% | Test Accur: 52.28% 
257500 Loss: 0.61 | Training Accur: 99.64% | Test Accur: 53.06% 
258000 Loss: 0.60 | Training Accur: 99.80% | Test Accur: 52.60% 
258500 Loss: 0.60 | Training Accur: 99.78% | Test Accur: 52.20% 
259000 Loss: 0.60 | Training Accur: 99.62% | Test Accur: 52.48% 
259500 Loss: 0.60 | Training Accur: 99.68% | Test Accur: 52.50% 
260000 Loss: 0.60 | Training Accur: 99.72% | Test Accur: 52.46% 
260500 Loss: 0.59 | Training Accur: 99.62% | Test Accur: 52.42% 
261000 Loss: 0.60 | Training Accur: 99.78% | Test Accur: 52.66% 
261500 Loss: 0.61 | Training Accur: 99.58% | Test Accur: 52.16% 
262000 Loss: 0.59 | Training Accur: 99.76% | Test Accur: 52.48% 
262500 Loss: 0.59 | Train

318500 Loss: 0.51 | Training Accur: 99.84% | Test Accur: 53.66% 
319000 Loss: 0.51 | Training Accur: 99.88% | Test Accur: 53.32% 
319500 Loss: 0.51 | Training Accur: 99.78% | Test Accur: 53.54% 
320000 Loss: 0.51 | Training Accur: 99.84% | Test Accur: 53.34% 
320500 Loss: 0.51 | Training Accur: 99.86% | Test Accur: 53.12% 
321000 Loss: 0.51 | Training Accur: 99.82% | Test Accur: 53.18% 
321500 Loss: 0.51 | Training Accur: 99.84% | Test Accur: 53.02% 
322000 Loss: 0.51 | Training Accur: 99.82% | Test Accur: 53.04% 
322500 Loss: 0.51 | Training Accur: 99.94% | Test Accur: 53.06% 
323000 Loss: 0.51 | Training Accur: 99.88% | Test Accur: 53.48% 
323500 Loss: 0.51 | Training Accur: 99.88% | Test Accur: 53.10% 
324000 Loss: 0.52 | Training Accur: 99.72% | Test Accur: 53.00% 
324500 Loss: 0.50 | Training Accur: 99.86% | Test Accur: 53.34% 
325000 Loss: 0.50 | Training Accur: 99.84% | Test Accur: 53.56% 
325500 Loss: 0.51 | Training Accur: 99.86% | Test Accur: 52.88% 
326000 Loss: 0.50 | Train

382000 Loss: 0.45 | Training Accur: 99.98% | Test Accur: 53.68% 
382500 Loss: 0.45 | Training Accur: 99.94% | Test Accur: 53.56% 
383000 Loss: 0.45 | Training Accur: 99.92% | Test Accur: 53.66% 
383500 Loss: 0.46 | Training Accur: 99.86% | Test Accur: 53.26% 
384000 Loss: 0.45 | Training Accur: 99.96% | Test Accur: 54.02% 
384500 Loss: 0.47 | Training Accur: 99.80% | Test Accur: 53.84% 
385000 Loss: 0.45 | Training Accur: 99.88% | Test Accur: 53.50% 
385500 Loss: 0.45 | Training Accur: 99.96% | Test Accur: 53.72% 
386000 Loss: 0.45 | Training Accur: 99.84% | Test Accur: 53.70% 
386500 Loss: 0.44 | Training Accur: 99.96% | Test Accur: 53.28% 
387000 Loss: 0.46 | Training Accur: 99.88% | Test Accur: 53.44% 
387500 Loss: 0.45 | Training Accur: 99.94% | Test Accur: 53.74% 
388000 Loss: 0.45 | Training Accur: 99.86% | Test Accur: 53.52% 
388500 Loss: 0.45 | Training Accur: 99.90% | Test Accur: 53.60% 
389000 Loss: 0.45 | Training Accur: 99.92% | Test Accur: 54.00% 
389500 Loss: 0.44 | Train

445500 Loss: 0.42 | Training Accur: 99.86% | Test Accur: 54.12% 
446000 Loss: 0.41 | Training Accur: 99.86% | Test Accur: 54.20% 
446500 Loss: 0.41 | Training Accur: 99.96% | Test Accur: 54.26% 
447000 Loss: 0.43 | Training Accur: 99.64% | Test Accur: 53.48% 
447500 Loss: 0.42 | Training Accur: 99.92% | Test Accur: 54.04% 
448000 Loss: 0.42 | Training Accur: 99.96% | Test Accur: 53.92% 
448500 Loss: 0.42 | Training Accur: 99.94% | Test Accur: 54.00% 
449000 Loss: 0.42 | Training Accur: 99.98% | Test Accur: 54.16% 
449500 Loss: 0.43 | Training Accur: 99.98% | Test Accur: 54.14% 
450000 Loss: 0.41 | Training Accur: 99.98% | Test Accur: 53.52% 
